# 04 QCPG Paraphrase
- Input: `03_augment_llm.csv`
- Output: `04_qcpg_paraphrase.csv`

In [1]:
import gc
import platform
import pandas as pd

import torch

from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

from tqdm.notebook import tqdm

2023-08-23 23:21:38.409805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 23:21:38.459772: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# If on mac default to mps, otherwise cuda, then cpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    # Check if on mac
    if platform.system() == "Darwin":
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
device

device(type='cuda', index=0)

## Load data

In [3]:
df = pd.read_csv("data/03_augment_llm.csv")
df

,context,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (born 3 December...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin (born December 3...,"Phatthira Saruritphong Phokin, who was born on..."
1,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator (Greek: Κλεοπάτρα θεά...,"Cleopatra VII Philopator, also known as Cleopa..."
2,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,Australia or the official name is Australian G...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,"Australia, officially known as the Australian ...","Australia, officially known as the Commonwealt..."
3,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,Thai Peace Day is the 16th of August every yea...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,Thai Peace Day is celebrated on August 16 ever...,The Thai Peace Day is celebrated annually on A...
4,นิวซีแลนด์ (อังกฤษ: New Zealand มาวรี: Aotearo...,New Zealand (English: New Zealand: Aotearoa me...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์: Aotearoa ห...,New Zealand (English: New Zealand; Maori: Aote...,"New Zealand, also known as Aotearoa or Niu Tir..."
...,...,...,...,...,...
7838,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,Black means the color of Shiva who drinks to p...,สีดำหมายถึงสีของพระอิศวรที่ดื่มเพื่อปกป้องมนุษ...,What does black symbolize within the context ...,"Black is the color associated with Shiva, who ..."
7839,ยัน ฟัน ไอก์ (Dutch: Jan van Eyck ราวก่อน ค.ศ....,"Dutch: Jan van Eyck, about 1395 - before July ...","ดัตช์: Jan Van Eyck, ประมาณ 1395 - ก่อนวันที่ ...","Jan van Eyck (about 1395-before July 9, 1441)...","Jan van Eyck, a Dutch artist born around 1395..."
7840,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,Bioshock Infinite is a videos of the first per...,BioShock Infinite เป็นวิดีโอของบุคคลแรกที่ถ่าย...,Bioshock Infinite is a first-person shooter v...,Bioshock Infinite is a first-person shooter v...
7841,ยู ซึง-โฮเดบิวต์เมื่อปี 1999 กับผลงานโฆษณาของ ...,Yoo Seung-Ho Debut in 1999 with the advertisem...,Yoo Seung-ho เปิดตัวในปี 1999 ด้วยการโฆษณาของ ...,Yoo Seung-Ho debuted in 1999 with an advertis...,Yoo Seung-Ho debuted in 1999 with an advertis...


In [7]:
pipe = pipeline('text2text-generation', model=f'ibm/qcpg-sentences', device=device)

In [5]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = int(len(string) * 0.5)
    return num_tokens

In [8]:
import nltk
from nltk.tokenize import sent_tokenize

def break_into_chunks(text: str, max_tokens: int = 1024) -> list:
    """Breaks the text into chunks that fit within the model's context using sentence segmentation."""
    chunks = []
    current_chunk = ""
    sentences = sent_tokenize(text)
    
    for sentence in sentences:
        temp_chunk = current_chunk + " " + sentence if current_chunk else sentence
        token_count = num_tokens_from_string(temp_chunk)
        
        if token_count <= max_tokens:
            current_chunk = temp_chunk
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    
    # Adding the last chunk if there's anything left
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def break_into_sentences(text: str) -> list:
    """Breaks the text into sentences using sentence segmentation."""
    sentences = sent_tokenize(text)
    return sentences

In [15]:
def run_augmentation(df, column_name, sem, lex, syn):
    df = df.copy(deep=True)
    prefix = f"COND_SEMANTIC_SIM_{int(sem*100)} COND_LEXICAL_DIV_{int(lex*100)} COND_SYNTACTIC_DIV_{int(syn*100)} "
    qcpg_augmented = []

    dataset = Dataset.from_pandas(df)

    # for out in tqdm(pipe(KeyDataset(dataset, column_name), batch_size=16), desc=f"Running QCPG", total=len(df)):
    #     qcpg_augmented.append(out[0]["generated_text"])
    
    for row in tqdm(dataset, desc=f"Running QCPG", total=len(df)):
        # chunks = break_into_chunks(row[column_name])
        chunks = break_into_sentences(row[column_name])
        finished_chunks = []
        # for chunk in chunks:
        #     out = pipe(prefix + chunk)
        #     finished_chunks.append(out[0]["generated_text"].strip())
        out = pipe([prefix + chunk for chunk in chunks])
        finished_chunks = [o["generated_text"].strip() for o in out]
        qcpg_augmented.append(" ".join(finished_chunks).strip())
        print(" ".join(finished_chunks).strip())

    gc.collect()
    
    return qcpg_augmented

In [16]:
settings_values = [0.2, 0.5, 0.8] 
columns = ["en_aug", "en_llm_gec_aug"]
for column in tqdm(columns, desc="Augmenting Columns"):
    for value in tqdm(settings_values, desc="Augmenting Column Values"):
        if column == "en_aug":
            target_name = f"en_qcpg_{value}_aug"
        else:
            target_name = f"en_qcpg_{value}_{'_'.join(column.split('_')[1:-1])}_aug"
        
        df[target_name] = run_augmentation(df, column, value, value, value)

Augmenting Columns:   0%|          | 0/2 [00:00<?, ?it/s]

Augmenting Column Values:   0%|          | 0/3 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/7843 [00:00<?, ?it/s]

In [ ]:
# Sanity check for no empty rows
assert df.isnull().sum().sum() == 0
df

In [ ]:
# Export and save QCPG
df.to_csv("data/04_paraphrase_qcpg.csv", index=False)

# Reformat output to make easier

In [1]:
import pandas as pd
import tqdm.autonotebook as tqdm

og_out = pd.read_csv('data/04_paraphrase_qcpg.csv')

/tmp/ipykernel_271589/1808653725.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdm


In [2]:
selected_columns = [x for x in og_out.columns if ("aug" in x and "th_aug" != x and "en_aug" != x)]
selected_columns

['en_llm_gec_aug',
 'en_llm_paraphrase_aug',
 'en_qcpg_0.2_aug',
 'en_qcpg_0.5_aug',
 'en_qcpg_0.8_aug',
 'en_qcpg_0.2_llm_gec_aug',
 'en_qcpg_0.5_llm_gec_aug',
 'en_qcpg_0.8_llm_gec_aug']

In [4]:
import os
import zipfile

# Export each column individually
for column in selected_columns:
    if not os.path.exists(f"data/04_paraphrase_qcpg_subsets"):
        os.makedirs(f"data/04_paraphrase_qcpg_subsets")
    og_out[["context", column]].to_csv(f'data/04_paraphrase_qcpg_subsets/04_paraphrase_qcpg_{column}.csv', index=False)

# Zip them all up with highest compression
with zipfile.ZipFile('data/04_paraphrase_qcpg_subsets.zip', 'w', zipfile.ZIP_DEFLATED, compresslevel=9) as zipf:
    for column in tqdm.tqdm(selected_columns):
        zipf.write(f'data/04_paraphrase_qcpg_subsets/04_paraphrase_qcpg_{column}.csv', arcname=f'04_paraphrase_qcpg_{column}.csv')

  0%|          | 0/8 [00:00<?, ?it/s]

# Merge Finished Augmentations

In [7]:
import pandas as pd
previous_augmentation = pd.read_csv('data/03_augment_llm.csv')
previous_augmentation

,context,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (born 3 December...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin (born December 3...,"Phatthira Saruritphong Phokin, who was born on..."
1,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator (Greek: Κλεοπάτρα θεά...,"Cleopatra VII Philopator, also known as Cleopa..."
2,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,Australia or the official name is Australian G...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,"Australia, officially known as the Australian ...","Australia, officially known as the Commonwealt..."
3,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,Thai Peace Day is the 16th of August every yea...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,Thai Peace Day is celebrated on August 16 ever...,The Thai Peace Day is celebrated annually on A...
4,นิวซีแลนด์ (อังกฤษ: New Zealand มาวรี: Aotearo...,New Zealand (English: New Zealand: Aotearoa me...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์: Aotearoa ห...,New Zealand (English: New Zealand; Maori: Aote...,"New Zealand, also known as Aotearoa or Niu Tir..."
...,...,...,...,...,...
7838,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,Black means the color of Shiva who drinks to p...,สีดำหมายถึงสีของพระอิศวรที่ดื่มเพื่อปกป้องมนุษ...,What does black symbolize within the context ...,"Black is the color associated with Shiva, who ..."
7839,ยัน ฟัน ไอก์ (Dutch: Jan van Eyck ราวก่อน ค.ศ....,"Dutch: Jan van Eyck, about 1395 - before July ...","ดัตช์: Jan Van Eyck, ประมาณ 1395 - ก่อนวันที่ ...","Jan van Eyck (about 1395-before July 9, 1441)...","Jan van Eyck, a Dutch artist born around 1395..."
7840,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,Bioshock Infinite is a videos of the first per...,BioShock Infinite เป็นวิดีโอของบุคคลแรกที่ถ่าย...,Bioshock Infinite is a first-person shooter v...,Bioshock Infinite is a first-person shooter v...
7841,ยู ซึง-โฮเดบิวต์เมื่อปี 1999 กับผลงานโฆษณาของ ...,Yoo Seung-Ho Debut in 1999 with the advertisem...,Yoo Seung-ho เปิดตัวในปี 1999 ด้วยการโฆษณาของ ...,Yoo Seung-Ho debuted in 1999 with an advertis...,Yoo Seung-Ho debuted in 1999 with an advertis...


In [11]:
original = pd.read_csv('../questions/data/01_prepare_dataset.csv')
original["source"].value_counts()

source
iapp      7226
tydiqa    4523
thaiqa    4051
xquad     1180
Name: count, dtype: int64

In [2]:
import zipfile
import os
# Finished augmentations have same filename and is stored in 04_paraphrase_qcpg_subsets_finished
# Unzip and merge all

with zipfile.ZipFile('data/04_paraphrase_qcpg_subsets_finished.zip', 'r') as zipf:
    zipf.extractall('data/')

# Get all the files
files = [x for x in os.listdir('data/04_paraphrase_qcpg_subsets_finished/') if x.endswith('.csv')]
files

['04_paraphrase_qcpg_en_llm_paraphrase_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.8_llm_gec_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.5_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.8_aug.csv',
 '04_paraphrase_qcpg_en_llm_gec_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.2_llm_gec_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.2_aug.csv',
 '04_paraphrase_qcpg_en_qcpg_0.5_llm_gec_aug.csv']

In [3]:
# Merge them all
merged_df = previous_augmentation.copy(deep=True) 

for file in files:
    df = pd.read_csv(f'data/04_paraphrase_qcpg_subsets_finished/{file}')

    cols_to_use = df.columns.difference(merged_df.columns).to_list()
    cols_to_use += ['context']

    merged_df = pd.merge(merged_df, df[cols_to_use], on='context', how='left')

merged_df

,context,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug,th_llm_paraphrase_aug,en_qcpg_0.8_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,en_qcpg_0.5_aug,th_qcpg_0.5_aug,en_qcpg_0.8_aug,th_qcpg_0.8_aug,th_llm_gec_aug,en_qcpg_0.2_llm_gec_aug,th_qcpg_0.2_llm_gec_aug,en_qcpg_0.2_aug,th_qcpg_0.2_aug,en_qcpg_0.5_llm_gec_aug,th_qcpg_0.5_llm_gec_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (born 3 December...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin (born December 3...,"Phatthira Saruritphong Phokin, who was born on...",Phatthira Saruritphong Phokin ซึ่งเกิดเมื่อวัน...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,The name of Phatthira Saruuritt Phokin (born o...,ชื่อของ Phatthira Saruuritt Phokin (เกิดในโรงเ...,Phatthira Saruutfokin (born on 3 December 1990...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,Phatthira Saruuritt Phokin (born on 3 December...,Phatthira Saruuritt Phokin (เกิดเมื่อวันที่ 3 ...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...
1,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator (Greek: Κλεοπάτρα θεά...,"Cleopatra VII Philopator, also known as Cleopa...",Cleopatra VII Philopator หรือที่รู้จักกันในชื่...,"Cleopatra VII, also known as Cleopatra, was Sh...",คลีโอพัตรา VII หรือที่รู้จักกันในชื่อคลีโอพัตร...,"Consequently, the last Greek family of the Tsa...",ดังนั้นครอบครัวกรีกคนสุดท้ายของราชวงศ์สตากา-เม...,"- 69 - T 69 - T Her dad's wolf mi, 12th. Orle...","- 69 - T 69 - T Wolf Mi พ่อของเธอ, 12th Orleti...",Cleopatra VII Philopator (กรีก: κλεοπάτραθεάφι...,"Cleopatra VII, also known as Cleopatra, was Th...",Cleopatra VII หรือที่รู้จักกันในชื่อ Cleopatra...,"- 69 - 69 - TABLE (LAUG Her dad's a wolf mi, ...","- 69 - 69 - โต๊ะ (Laug พ่อของเธอ A Wolf Mi, 12...","Cleopatra VII, also known as Cleopatra, was So...",Cleopatra VII หรือที่รู้จักกันในชื่อคลีโอพัตรา...
2,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,Australia or the official name is Australian G...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,"Australia, officially known as the Australian ...","Australia, officially known as the Commonwealt...",ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"Australia, which is formally called Australia,...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"The Australian group (Australian), Tae-Samanth...","กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...","Australia, or officially Australia, is a count...",ออสเตรเลียหรือออสเตรเลียอย่างเป็นทางการเป็นประ...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"Australia, which is formally called Australia,...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"The Australian group (Australian), Tae-Samanth...","กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...","The Australian group, also known as the Austra...",กลุ่มออสเตรเลียหรือที่รู้จักกันในชื่อกลุ่มออสเ...
3,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,Thai Peace Day is the 16th of August every yea...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,Thai Peace Day is celebrated on August 16 ever...,The Thai Peace Day is celebrated annually on A...,วันสันติภาพไทยมีการเฉลิมฉลองเป็นประจำทุกปีในวั...,The Thai peace day is celebrated on 16th ofaug...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองเมื่อวันท...,"The Thai peace day is the 16th ofaugust 1945, ...",วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,"Thai peace day is celebrated on 16th ofaugust,...",วันสันติภาพไทยมีการเฉลิมฉลองในวันที่ 16 ของ AU...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองวันที่ 16...,The Thai peace day is celebrated on the 16th o...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองในวันที่ ...,"The Thai pea

In [4]:
# Assert length is the same
assert len(merged_df) == len(previous_augmentation)

In [5]:
import hashlib
# Hash context column as id

def hash_string(string: str) -> str:
    """Hashes a string using sha256."""
    return hashlib.sha256(string.encode()).hexdigest()

merged_df['id'] = merged_df['context'].apply(hash_string)
merged_df

,context,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug,th_llm_paraphrase_aug,en_qcpg_0.8_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,en_qcpg_0.5_aug,th_qcpg_0.5_aug,en_qcpg_0.8_aug,th_qcpg_0.8_aug,th_llm_gec_aug,en_qcpg_0.2_llm_gec_aug,th_qcpg_0.2_llm_gec_aug,en_qcpg_0.2_aug,th_qcpg_0.2_aug,en_qcpg_0.5_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,id
0,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (born 3 December...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin (born December 3...,"Phatthira Saruritphong Phokin, who was born on...",Phatthira Saruritphong Phokin ซึ่งเกิดเมื่อวัน...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,The name of Phatthira Saruuritt Phokin (born o...,ชื่อของ Phatthira Saruuritt Phokin (เกิดในโรงเ...,Phatthira Saruutfokin (born on 3 December 1990...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,Phatthira Saruuritt Phokin (born on 3 December...,Phatthira Saruuritt Phokin (เกิดเมื่อวันที่ 3 ...,Phatthira Saruutfokin (born 3 December 1990) i...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...
1,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator (Greek: Κλεοπάτρα θεά...,"Cleopatra VII Philopator, also known as Cleopa...",Cleopatra VII Philopator หรือที่รู้จักกันในชื่...,"Cleopatra VII, also known as Cleopatra, was Sh...",คลีโอพัตรา VII หรือที่รู้จักกันในชื่อคลีโอพัตร...,"Consequently, the last Greek family of the Tsa...",ดังนั้นครอบครัวกรีกคนสุดท้ายของราชวงศ์สตากา-เม...,"- 69 - T 69 - T Her dad's wolf mi, 12th. Orle...","- 69 - T 69 - T Wolf Mi พ่อของเธอ, 12th Orleti...",Cleopatra VII Philopator (กรีก: κλεοπάτραθεάφι...,"Cleopatra VII, also known as Cleopatra, was Th...",Cleopatra VII หรือที่รู้จักกันในชื่อ Cleopatra...,"- 69 - 69 - TABLE (LAUG Her dad's a wolf mi, ...","- 69 - 69 - โต๊ะ (Laug พ่อของเธอ A Wolf Mi, 12...","Cleopatra VII, also known as Cleopatra, was So...",Cleopatra VII หรือที่รู้จักกันในชื่อคลีโอพัตรา...,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...
2,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,Australia or the official name is Australian G...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,"Australia, officially known as the Australian ...","Australia, officially known as the Commonwealt...",ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"Australia, which is formally called Australia,...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"The Australian group (Australian), Tae-Samanth...","กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...","Australia, or officially Australia, is a count...",ออสเตรเลียหรือออสเตรเลียอย่างเป็นทางการเป็นประ...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"Australia, which is formally called Australia,...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"The Australian group (Australian), Tae-Samanth...","กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...","The Australian group, also known as the Austra...",กลุ่มออสเตรเลียหรือที่รู้จักกันในชื่อกลุ่มออสเ...,287fa29b965a1ba84011bb12ffafb5194984a955d851eb...
3,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,Thai Peace Day is the 16th of August every yea...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,Thai Peace Day is celebrated on August 16 ever...,The Thai Peace Day is celebrated annually on A...,วันสันติภาพไทยมีการเฉลิมฉลองเป็นประจำทุกปีในวั...,The Thai peace day is celebrated on 16th ofaug...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองเมื่อวันท...,"The Thai peace day is the 16th ofaugust 1945, ...",วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,"Thai peace day is celebrated on 16th ofaugust,...",วันสันติภาพไทยมีการเฉลิมฉลองในวันที่ 16 ของ AU...,วันแห่งควา

In [6]:
# Export results
merged_df.to_parquet('data/05_augment_thai_input.parquet', index=False)